# Exploring Data from 311 Service Calls in Chicago

*[Repeated from last time: On the [Chicago Data Portal](https://data.cityofchicago.org/), the city of Chicago provides a wealth of data about the city and its governance.  One of the many datasets available catalogs 311 service calls (311 is a telephone number through which the city provides non-emergency services), and of the many collections of data within that broader set, we can find a list of [service requests regarding abandoned vehicles](https://data.cityofchicago.org/Service-Requests/311-Service-Requests-Abandoned-Vehicles-No-Duplica/atid-bgws) collected since 2011.]*

In this exercise, you will work through guided steps to further explore and analyze the 311 data.  Much of this will be similar to work done in the previous class (you can see it in `in_class_5.2_worked.ipynb`).

Look for `#TODO` comments in cells for places where you should write code.

This exercise is structured so that you will create variables with partially-processed data and use those variables in later cells to do further analysis.  Remember: once you create a variable in one cell, it can be used in any other cell.

***

The data is provided in a file that has been copied into this directory: `311_Service_Requests_-_Abandoned_Vehicles_-_No_Duplicates.csv`.  First, we have a bit of code to read the contents of the file into a single string:

In [1]:
with open("311_Service_Requests_-_Abandoned_Vehicles_-_No_Duplicates.csv") as f:
    file_contents = f.read()

After executing that cell, `file_contents` will be a single string containing the complete contents of the file.

Verify that it loaded correctly by printing the first 1000 characters of the string.  (Don't print the whole thing!  It's far too long.)

In [2]:
file_contents[:1000]

'Creation Date,Status,Completion Date,Service Request Number,Type of Service Request,License Plate,Vehicle Make/Model,Vehicle Color,Current Activity,Most Recent Action,How Many Days Has the Vehicle Been Reported as Parked?,Street Address,ZIP Code,Ward,Police District,Community Area,SSA\n01/01/2011,Completed,01/05/2011,11-00001976,Abandoned Vehicle Complaint,H924236,Ford,White,,,60,6059 S KOMENSKY AVE,60629,13,8,65,3\n01/01/2011,Completed,01/05/2011,11-00002291,Abandoned Vehicle Complaint,810 LYB    WISCONSIN PLATES,Mercury,Green,,,,4651 S WASHTENAW AVE,60632,12,9,58,\n01/01/2011,Completed,01/05/2011,11-00002696,Abandoned Vehicle Complaint,368M783,Buick,Gold,,,10,6200 S MASSASOIT AVE,60638,13,8,64,\n01/01/2011,Completed,01/05/2011,11-00003094,Abandoned Vehicle Complaint,000000000,Dodge,White,,,30,5816 S ALBANY AVE,60629,14,8,63,59\n01/01/2011,Completed,01/05/2011,11-00003456,Abandoned Vehicle Complaint,TEXAS PLATE  -  SMALL FLATBED HITCH TRAILER  -  MISSING TIRES,,Black,,,,4559 S KEELER

***

The file contains 160,000+ rows, each one on its own line of the file.  To start, we need to separate the `file_contents` string into individual lines.  Make a list called `lines` here by splitting `file_contents` on the newline character (`'\n'`).

In [3]:
lines = file_contents.split('\n')

Verify that the list contains individual lines by printing the first three elements in the list.

In [4]:
lines[:3]

['Creation Date,Status,Completion Date,Service Request Number,Type of Service Request,License Plate,Vehicle Make/Model,Vehicle Color,Current Activity,Most Recent Action,How Many Days Has the Vehicle Been Reported as Parked?,Street Address,ZIP Code,Ward,Police District,Community Area,SSA',
 '01/01/2011,Completed,01/05/2011,11-00001976,Abandoned Vehicle Complaint,H924236,Ford,White,,,60,6059 S KOMENSKY AVE,60629,13,8,65,3',
 '01/01/2011,Completed,01/05/2011,11-00002291,Abandoned Vehicle Complaint,810 LYB    WISCONSIN PLATES,Mercury,Green,,,,4651 S WASHTENAW AVE,60632,12,9,58,']

***

Let's try to find the average number of days a vehicle was reported parked (before being reported, I assume).  One of the column headers is "How Many Days Has the Vehicle Been Reported as Parked?"  We'll need to get the value in this column from each row.

If you look at the header row (the first line of the file), you can see that this column will be at index **10** (given that indexes start counting from 0).

We can define a function to get the value in that column from any given line.  Specifically, the function should:
1. Have a single parameter, `line`, that will be a single line of the file as a string.
2. Split the `line` parameter on the `','` character.
3. Some lines in the file aren't complete, so we have to filter out lines without an index 10.
   * Filter out any line that has fewer than 11 entries (remember: index 10 would be the 11th entry, since counting starts at 0): return the special value `None` in that case.
   * This has to happen *before* the next step, because trying to access index 10 if it doesn't exist would cause an `IndexError` exception and crash the program.
4. Get the value in the resulting list at index 10.
5. Some lines will have messy data, so a bit more filtering is needed:
   * Filter out any value that is an empty string : return `None` in that case.
   * Filter out any value that has more than 10 characters: return `None` in that case.
6. For any values that you haven't filtered out, convert the string to an integer.
7. Return the integer value.

Name the function `get_days()`.  You can look at the `get_color()` function in `in_class_5.2_worked.py` to see how we started a similar function there.

In [5]:
def get_days(line):
    # split line on commas
    entries = line.split(',')
    
    # return None if no value at index 10
    if len(entries) < 11:
        return None
    
    # get value at index 10
    days = entries[10]
    
    # return None for empty strings
    if days == "":
        return None
    
    # return None for string longer than 10 chars
    if len(days) > 10:
        return None
    
    # convert value to an integer
    days_int = int(days)
    
    # return integer value
    return days_int

To test this function, call it and pass it the string below as an argument.  It should return `14`.

In [6]:
test_line = "01/02/2012,Completed,01/23/2012,12-00003795,Abandoned Vehicle Complaint,L701065,Chevrolet,Gray,FVI - Outcome,Vehicle was moved from original address requested,14,10240 S EBERHART AVE,60628,9,5,49,41"

get_days(test_line)

14

***

To calculate the average number of days parked, we will need a list of all of the "days parked" values.

Use the `get_days()` function defined above in a for loop to get the "days parked" value from every line in the file and **append** each to a list called `days_list`.  (See the textbook [Section 5.2.5](https://snakebear.science/05-StringsListsAndFiles/Lists.html#index-35) for a quick example of the `.append()` list method.)

Print the first several values of `days_list` to make sure it looks reasonable.  Every value should either be `None` or an integer.

In [7]:
days_list = []

for line in lines:
    days = get_days(line)
    days_list.append(days)

days_list[:10]

[None, 60, None, 10, 30, None, 7, 15, 9, 30]

***

If you try to calculate the average of `days_list`, you will find that it contains some invalid values.  Some entries will be `None` because of how `get_days()` treated empty strings.  And some values will be strangely large while others are negative.  It is unlikely a car was parked for more than 1000 days, for example, or for -100 days, but those values show up in the data!

Before calculating a correct average, then, we need to remove invalid values.

First: `None` values.

**Important:** Once we have created one list, it is best not to change it, because we might want to reuse that list later.  Because we are going to modify the list by removing values, we should make a **copy** of `days_list` into a new list named `days_cleaned`, and remove invalid values from the `days_cleaned` list.

The code below removes `None` values (it's a bit more complex than is worth figuring out during our limited class time):
1. First, it makes a copy of `days_list` in a new variable named `days_cleaned`.
2. Next, it removes any `None` values from `days_cleaned` using an index variable `i` and a loop to iterate through the whole list.
3. Then, it prints the lengths of `days_list` and `days_cleaned` to make sure a reasonable number of values has been removed ("reasonable" here is more than 0 and less than all of the values).

In [8]:
# Copy days_list to days_cleaned
days_cleaned = days_list[:]

# Go through the list, deleting any None values
i = 0
while i < len(days_cleaned):
    if days_cleaned[i] is None:
        del days_cleaned[i]
    else:
        i = i + 1

# Print the original length and the "cleaned" length
print("Original:", len(days_list))
print("Cleaned:", len(days_cleaned))
        
# Print the first ten values of the "cleaned" list.
print(days_cleaned[:10])

Original: 161435
Cleaned: 146862
[60, 10, 30, 7, 15, 9, 30, 30, 21, 45]


Next, we need to remove negative and overly-large values from `days_cleaned`.

One way to do this is to:
1. *Sort* the list using the `.sort()` list method.
2. Look at the values at the beginning and end of the list using indexing or slicing to explore the smallest and largest values.
3. Go *back* and modify the `get_days()` function to exclude values outside of a range we choose.
4. Re-run the cells from when `get_days()` is defined to here to re-do the analysis with those values excluded.

Use the cell below to sort `days_cleaned` and look at the values at its largest and smallest values, then modify and re-define `get_days()` based on what you see.

When you come back and execute this again after changing `get_days()`, you should see more reasonable values at the ends of the sorted list.

In [9]:
days_cleaned.sort()

print(days_cleaned[:10])    # first 10 values
print(days_cleaned[-100:])  # last 100 values

[-523454546, -762, -720, -690, -690, -690, -600, -500, -366, -365]
[1500, 1500, 1500, 1515, 1515, 1600, 1600, 1635, 1800, 1800, 1800, 1815, 1825, 1825, 1825, 1825, 1825, 1825, 1825, 1825, 1825, 1825, 1825, 2000, 2000, 2000, 2000, 2000, 2003, 2013, 2100, 2150, 2180, 2180, 2190, 2555, 2555, 2555, 2555, 2900, 2920, 2920, 2920, 3000, 3003, 3650, 3650, 3650, 3650, 3650, 4380, 4380, 5475, 9999, 10000, 10767, 10915, 11111, 21312, 30000, 98939, 99999, 99999, 99999, 99999, 99999, 99999, 99999, 99999, 99999, 99999, 99999, 100617, 110114, 110410, 112111, 116804, 122510, 130820, 171626, 605887, 800000, 1000000, 1091649, 1371356, 2884927, 3072016, 4012016, 6676478, 8042016, 9999999, 10000000, 12152014, 1700409381, 3123157293, 3123157293, 7736804250, 7738515816, 7738669204, 9999999999]


***

Now we have a list of cleaned, correct values.  Calculate the average!

You can do this a few different ways:
1. Use a for loop.
2. Use the `sum()` and `len()` functions.
3. Use the `statistics.mean()` function from the `statistics` module [[documentation](https://docs.python.org/3/library/statistics.html#statistics.mean)].

In [10]:
import statistics

average = statistics.mean(days_cleaned)

# In this worked example, we have not yet gone back to filter negative and overly-large values
# from days_cleaned, so this average (277175.3456...) is not reasonable yet.
print(average)

277175.3456782558


***

That's the main goal of this exercise.  If you want to go further, a next step might be to modify `get_days()` to take an additional parameter and return `None` for any lines that *don't* contain that parameter.  Then you could call, for example, `get_days(line, 'Chevrolet')` and get back numbers for only the Chevrolets (or any other make you specify).  After that modification, you can re-run the rest of the analysis, putting a different make into the argument for `get_days()` each time, to look for differences in how long different makes of cars were reported abandoned.  Have Nissans been sitting out there longer than other cars?  You can find out!